In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Imported Libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections


# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

/Users/songheelee/opt/anaconda3/envs/xgb/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/songheelee/opt/anaconda3/envs/xgb/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/songheelee/opt/anaconda3/envs/xgb/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/songheelee/opt/anacon

In [12]:
df = pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [13]:
# The classes are heavily skewed we need to solve this issue later.
print('No Frauds', round(df['Class'].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('Frauds', round(df['Class'].value_counts()[1]/len(df) * 100,2), '% of the dataset')

No Frauds 99.83 % of the dataset
Frauds 0.17 % of the dataset


In [14]:
print(df['Class'].value_counts()[0])
print(df['Class'].value_counts()[1])

284315
492


### 스케일링

In [15]:
# Since most of our data has already been scaled we should scale the columns that are left to scale (Amount and Time)
from sklearn.preprocessing import StandardScaler, RobustScaler

# RobustScaler is less prone to outliers.

std_scaler = StandardScaler()
rob_scaler = RobustScaler()

df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))

df.drop(['Time','Amount'], axis=1, inplace=True)

In [16]:
scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']

df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)

# Amount and Time are Scaled!

df.head()

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.783274,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.269825,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.983721,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.418291,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.670579,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


### 이상치 제거

In [17]:
"""
column = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
        'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
        'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28' 
         ]

column = ['V2', 'V4', 'V11', 'V19', 'V12', 'V14', 'V17', 'V25']



column = ['V4', 'V6', 'V9', 'V10', 'V11',
        'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19',
        'V21', 'V22', 'V24', 'V26', 'V28' ]
        
"""

column = ['V2', 'V4', 'V11', 'V19', 'V12', 'V14', 'V17', 'V25']
for idx in column :
    fraud = df[idx].loc[df['Class'] == 1].values

    q25, q75 = np.percentile(fraud, 25), np.percentile(fraud, 75)
    print('Quartile 25: {} | Quartile 75: {}'.format(q25, q75))

    iqr = q75 - q25
    print('iqr: {}'.format(iqr))

    #cut_off = iqr * 1.5
    cut_off = iqr * 2.0

    lower, upper = q25 - cut_off, q75 + cut_off
    print('Cut Off: {}'.format(cut_off))
    print('{} Lower: {}'.format(idx, lower))
    print('{} Upper: {}'.format(idx, upper))

    outliers = [x for x in fraud if x < lower or x > upper]
    print('Feature {} Outliers for Fraud Cases: {}'.format(idx, len(outliers)))
    print('{} outliers:{}'.format(idx, outliers))

    df = df.drop(df[(df[idx] > upper) | (df[idx] < lower)].index)
    print('----' * 44)


Quartile 25: 1.1882258198658224 | Quartile 75: 4.9712573249056025
iqr: 3.78303150503978
Cut Off: 7.56606301007956
V2 Lower: -6.377837190213738
V2 Upper: 12.537320334985163
Feature V2 Outliers for Fraud Cases: 37
V2 outliers:[12.6521968313004, 12.9305051249875, 13.208904284417601, 13.4873857909274, 13.7659421584186, 14.044566781510598, 14.323253809723301, 14.6019980426299, 15.3658043803315, 15.598192662555402, 15.876922987953598, 16.1557014298057, 16.4345245512223, 16.7133892350242, -6.976420007546411, 12.785970638297998, 12.785970638297998, 12.785970638297998, 12.785970638297998, 12.785970638297998, 12.785970638297998, 16.497471901867, 16.497471901867, 16.497471901867, 16.497471901867, 12.572117853538499, 12.864988563015, 14.7063346696674, 15.53613332478, 16.6978316913154, 19.167239010306197, 21.4672029942752, 22.0577289904909, -7.15904171709445, -7.4490151587267395, -7.19697963053735, -8.402153677689151]
---------------------------------------------------------------------------------

In [18]:
print(df['Class'].value_counts()[0])
print(df['Class'].value_counts()[1])

281341
443


### Train & Test 분리

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

print('No Frauds', round(df['Class'].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('Frauds', round(df['Class'].value_counts()[1]/len(df) * 100,2), '% of the dataset')

X = df.drop('Class', axis=1)
y = df['Class']

sss = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]

# We already have X_train and y_train for undersample data thats why I am using original to distinguish and to not overwrite these variables.
# original_Xtrain, original_Xtest, original_ytrain, original_ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the Distribution of the labels


# Turn into an array
Xtrain = original_Xtrain.values
Xtest = original_Xtest.values
ytrain = original_ytrain.values
ytest = original_ytest.values

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)
print('-' * 100)

print('Label Distributions: \n')
print(train_counts_label/ len(original_ytrain))
print(test_counts_label/ len(original_ytest))

No Frauds 99.84 % of the dataset
Frauds 0.16 % of the dataset
Train: [ 41718  41720  42179 ... 281781 281782 281783] Test: [    0     1     2 ... 56402 56403 56404]
Train: [     0      1      2 ... 281781 281782 281783] Test: [ 41718  41720  42179 ... 112746 112747 112748]
Train: [     0      1      2 ... 281781 281782 281783] Test: [ 82387  82625  82743 ... 169118 169119 169120]
Train: [     0      1      2 ... 281781 281782 281783] Test: [149606 149613 149619 ... 225441 225442 225443]
Train: [     0      1      2 ... 225441 225442 225443] Test: [213866 213896 216306 ... 281781 281782 281783]
----------------------------------------------------------------------------------------------------
Label Distributions: 

[0.99842522 0.00157478]
[0.9984385 0.0015615]


### 분류기 결과

In [20]:
# Wow our scores are getting even high scores even when applying cross validation.
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.metrics import roc_auc_score

#original_Xtrain original_ytrain

classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "RandomForestClassifier": RandomForestClassifier()
}


In [21]:

for key, classifier in classifiers.items():
    classifier.fit(Xtrain, ytrain)
    
    y_pred = classifier.predict(Xtest)
    print("Classifiers: {}".format(classifier.__class__.__name__))
    print('Accuracy Score: {:.2f}'.format(accuracy_score(ytest, y_pred)))
    print('Recall Score: {:.2f}'.format(recall_score(ytest, y_pred)))
    print('Precision Score: {:.2f}'.format(precision_score(ytest, y_pred)))
    print('F1 Score: {:.2f}'.format(f1_score(ytest, y_pred)))
    print('ROC AUC score: {:.2f}'.format(roc_auc_score(ytest, y_pred)))
    print('---' * 45)


Classifiers: LogisticRegression
Accuracy Score: 1.00
Recall Score: 0.57
Precision Score: 0.94
F1 Score: 0.71
ROC AUC score: 0.78
---------------------------------------------------------------------------------------------------------------------------------------
Classifiers: KNeighborsClassifier
Accuracy Score: 1.00
Recall Score: 0.72
Precision Score: 0.98
F1 Score: 0.83
ROC AUC score: 0.86
---------------------------------------------------------------------------------------------------------------------------------------
Classifiers: SVC
Accuracy Score: 1.00
Recall Score: 0.59
Precision Score: 1.00
F1 Score: 0.74
ROC AUC score: 0.80
---------------------------------------------------------------------------------------------------------------------------------------
Classifiers: RandomForestClassifier
Accuracy Score: 1.00
Recall Score: 0.68
Precision Score: 1.00
F1 Score: 0.81
ROC AUC score: 0.84
-------------------------------------------------------------------------------------